# clanker robot arm

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import serial
import time
from collections import deque


### constants initialization

In [ ]:
SERIAL_PORT = 'COM3'
BAUD_RATE = 115200
CAMERA_INDEX = 0
SMOOTHING_WINDOW = 7 


### This creates the serial comms channel between python and arduino through CVzone library and has code that accounts for error

In [ ]:
try:
    arduino = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=1)
    time.sleep(2)
    print(f"✓ Arduino connected on {SERIAL_PORT}")
except Exception as e:
    print(f"✗ Arduino error: {e}")
    exit(1)

#CVZone hand detector
detector = HandDetector(detectionCon=0.7, maxHands=1)

#webcam
cap = cv2.VideoCapture(CAMERA_INDEX)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

if not cap.isOpened():
    print("✗ Webcam error")
    exit(1)

#### Mapping out the landmarks over the hand, in relation to the computer vision. this code measures the bending of each finger. 
Index → Servo A

Middle → Servo D

Ring → Servo B

Pinky → Servo C

Thumb → Servo E 

this organises each finger based on the servo motor its connected to. additionaly i added a buffer that helps remove jitter and make sure there isnt random servo motor vibration

then with the finger id part of the code, using math to measure from the base knuckle to the finger (4 landmarks for each finger)

dist = sqrt((x2-x1)² + (y2-y1)²) this specific calculation determines a large or short distance ( corresponding to open or bent finger)

at the end def send_to_arduino(bend_dict): will send the data over to the arduino in this format A,B,C,D,E\n

i also added error handling to prevent a crash if the arduino disconnects or smthn like that

In [ ]:
# Finger Mapping (NOW WITH THUMB)
FINGERS = {
    'thumb': 0,
    'index': 1,
    'middle': 2,
    'ring': 3,
    'pinky': 4
}

SERVO_MAP = {
    'index': 'A',
    'middle': 'D',
    'ring': 'B',
    'pinky': 'C',
    'thumb': 'E'  
}

# Calibration
calib_state = 'collecting_open'
calib_frames_needed = 60
calib_frame_count = 0

calib_data = {
    'thumb': {'open_samples': [], 'closed_samples': [], 'open': None, 'closed': None},
    'index': {'open_samples': [], 'closed_samples': [], 'open': None, 'closed': None},
    'middle': {'open_samples': [], 'closed_samples': [], 'open': None, 'closed': None},
    'ring': {'open_samples': [], 'closed_samples': [], 'open': None, 'closed': None},
    'pinky': {'open_samples': [], 'closed_samples': [], 'open': None, 'closed': None}
}

smooth_buffers = {f: deque(maxlen=SMOOTHING_WINDOW) for f in FINGERS.keys()}


def get_finger_distance(lmList, finger_name):
    """Get distance from fingertip to base"""
    finger_id = FINGERS[finger_name]
    
    # CVZone landmarks
    base_idx = finger_id * 4 + 1
    tip_idx = finger_id * 4 + 4
    
    if len(lmList) > tip_idx:
        base = lmList[base_idx]
        tip = lmList[tip_idx]
        
        dist = np.sqrt((tip[0] - base[0])**2 + (tip[1] - base[1])**2)
        return dist
    return 0

def normalize_bend(dist, open_dist, closed_dist):
    if open_dist == closed_dist:
        return 0.0
    norm = (open_dist - dist) / (open_dist - closed_dist)
    return np.clip(norm, 0.0, 1.0)

def smooth_value(finger, value):
    smooth_buffers[finger].append(value)
    return sum(smooth_buffers[finger]) / len(smooth_buffers[finger])

def send_to_arduino(bend_dict):
    # Format: "bendA,bendB,bendC,bendD,bendE\n"
    servo_values = {
        'A': bend_dict.get('index', 0.0),
        'B': bend_dict.get('ring', 0.0),
        'C': bend_dict.get('pinky', 0.0),
        'D': bend_dict.get('middle', 0.0),
        'E': bend_dict.get('thumb', 0.0)  # NEW
    }
    cmd = f"{servo_values['A']:.3f},{servo_values['B']:.3f},{servo_values['C']:.3f},{servo_values['D']:.3f},{servo_values['E']:.3f}\n"
    try:
        arduino.write(cmd.encode())
    except:
        pass

### Over here i created a little dashboard to show the calibration status and the finger bends in real time. in the calibration it requires user to hold hand open and closed for a couple of seconds before they can begin
 this code if key == 'thumb':
    pwm = 1200 + int(bend * 600)
else:
    pwm = 500 + int(bend * 2000) converts the bend into PWM which is what the arduino outputs 


hands, frame = detector.findHands(frame, draw=True, flipType=False)

this crucial part is what actually detects the hand landmarks through mediapipe (hands is what has the landmark data that for logic)


with a quick function  this creates the bend bars that show in real time the bending of each finger

fill = int(bend * bar_w)
cv2.rectangle(...) 

and lastly i added user controls like R for recalibration and Q to quit

In [ ]:
def draw_ui(frame, bend_dict, calib_state, calib_progress, hand_detected, fps):
    h, w = frame.shape[:2]
    
    overlay = frame.copy()
    cv2.rectangle(overlay, (0, 0), (w, 270), (25, 25, 25), -1)  # Taller for 5 fingers
    cv2.addWeighted(overlay, 0.85, frame, 0.15, 0, frame)
    
    cv2.putText(frame, "ROBOTIC HAND (5 FINGERS)", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 2)
    
    cv2.putText(frame, f"FPS: {fps:.0f}", (w-130, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    if calib_state == 'collecting_open':
        status = f"HOLD OPEN HAND... {calib_progress}%"
        color = (0, 165, 255)
    elif calib_state == 'collecting_closed':
        status = f"HOLD CLOSED FIST... {calib_progress}%"
        color = (0, 165, 255)
    else:
        status = "CALIBRATED - CONTROLLING ROBOT"
        color = (0, 255, 0) if hand_detected else (0, 0, 255)
    
    cv2.putText(frame, status, (20, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
    
    if calib_state != 'calibrated':
        bar_x, bar_y = 20, 100
        bar_w, bar_h = 400, 20
        cv2.rectangle(frame, (bar_x, bar_y), (bar_x+bar_w, bar_y+bar_h), (60,60,60), -1)
        fill = int((calib_progress/100) * bar_w)
        cv2.rectangle(frame, (bar_x, bar_y), (bar_x+fill, bar_y+bar_h), (0,255,255), -1)
        cv2.rectangle(frame, (bar_x, bar_y), (bar_x+bar_w, bar_y+bar_h), (255,255,255), 2)
    
    if calib_state == 'calibrated':
        y = 120
        fingers_display = [
            ('Thumb (E→D8)',  'thumb',  (255, 150, 50)),   # NEW
            ('Index (A→D3)',  'index',  (100, 200, 255)),
            ('Middle (D→D9)', 'middle', (100, 255, 100)),
            ('Ring (B→D5)',   'ring',   (255, 200, 100)),
            ('Pinky (C→D6)',  'pinky',  (255, 100, 200))
        ]
        
        for label, key, color in fingers_display:
            bend = bend_dict.get(key, 0.0)
            
            # Thumb uses different range
            if key == 'thumb':
                pwm = 1200 + int(bend * 600)  # 1200-1800 for MG90
            else:
                pwm = 500 + int(bend * 2000)   # 500-2500 for MG996R
            
            text = f"{label}: {bend:.2f} → {pwm}µs"
            cv2.putText(frame, text, (20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,255), 1)
            
            bar_x = 350
            bar_w = 300
            bar_h = 16
            cv2.rectangle(frame, (bar_x, y-13), (bar_x+bar_w, y-13+bar_h), (60,60,60), -1)
            fill = int(bend * bar_w)
            cv2.rectangle(frame, (bar_x, y-13), (bar_x+fill, y-13+bar_h), color, -1)
            cv2.rectangle(frame, (bar_x, y-13), (bar_x+bar_w, y-13+bar_h), (255,255,255), 2)
            
            y += 26
    
    cv2.putText(frame, "Q: Quit  |  R: Recalibrate", (20, h-20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (180,180,180), 1)
    
    return frame

# Main Loop
frame_count = 0
fps = 0
fps_time = time.time()

print("✓ Starting...\n")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    
    hands, frame = detector.findHands(frame, draw=True, flipType=False)
    
    bend_values = {}
    hand_detected = False
    

### This final chunk of code is a continuation to the markdown before which controls calibration and smoothens values from the live bend of the finger through the land mark tracking

the 'if hands:' conditional only runs the tracking if there is a detected hand within the webcam 

for fname in FINGERS.keys():
    calib_data[fname]['closed'] = np.mean(calib_data[fname]['closed_samples'])
calib_state = 'calibrated'
print("✓ Calibrated! Controlling 5 fingers...\n")

this is the code for the finishe calibration of the closed hand. which shows visually the arm is fully calibrated and begins hand control. 

each finger frame: 

dist = get_finger_distance(lmList, fname)
open_d = calib_data[fname]['open']
closed_d = calib_data[fname]['closed']

if open_d and closed_d:
    bend = normalize_bend(dist, open_d, closed_d)
    smooth = smooth_value(fname, bend)
    bend_values[fname] = smooth

this code runs a pipeline that measures the current finger distance, fetches that finger’s calibrated open/closed distances and convert to bend [0,1] or in other words open or closed and stores the result in bend_values

this is lastly then sent to arduino through this code send_to_arduino(bend_values)

i later decided to add this conditional 

if frame_count % 2 == 0:

that sends the data to arduino every 2 frames so i could reduce serial bandwith usage

and after just some final code and controls to quit the software and drawing the user interface



In [ ]:
if hands:
        hand_detected = True
        hand = hands[0]
        lmList = hand['lmList']
        
        if calib_state in ['collecting_open', 'collecting_closed']:
            for fname in FINGERS.keys():
                dist = get_finger_distance(lmList, fname)
                
                if calib_state == 'collecting_open':
                    calib_data[fname]['open_samples'].append(dist)
                else:
                    calib_data[fname]['closed_samples'].append(dist)
            
            calib_frame_count += 1
            
            if calib_frame_count >= calib_frames_needed:
                if calib_state == 'collecting_open':
                    for fname in FINGERS.keys():
                        calib_data[fname]['open'] = np.mean(calib_data[fname]['open_samples'])
                    calib_state = 'collecting_closed'
                    calib_frame_count = 0
                    print("✓ Open calibrated! Close fist...")
                
                elif calib_state == 'collecting_closed':
                    for fname in FINGERS.keys():
                        calib_data[fname]['closed'] = np.mean(calib_data[fname]['closed_samples'])
                    calib_state = 'calibrated'
                    print("✓ Calibrated! Controlling 5 fingers...\n")
        
        elif calib_state == 'calibrated':
            for fname in FINGERS.keys():
                dist = get_finger_distance(lmList, fname)
                open_d = calib_data[fname]['open']
                closed_d = calib_data[fname]['closed']
                
                if open_d and closed_d:
                    bend = normalize_bend(dist, open_d, closed_d)
                    smooth = smooth_value(fname, bend)
                    bend_values[fname] = smooth
            
            if frame_count % 2 == 0:
                send_to_arduino(bend_values)
    
    if calib_state in ['collecting_open', 'collecting_closed']:
        progress = int((calib_frame_count / calib_frames_needed) * 100)
    else:
        progress = 100
    
    if frame_count % 10 == 0:
        now = time.time()
        fps = 10.0 / (now - fps_time)
        fps_time = now
    
    frame = draw_ui(frame, bend_values, calib_state, progress, hand_detected, fps)
    
    frame_count += 1
    cv2.imshow('Robotic Hand Control', frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('r'):
        calib_state = 'collecting_open'
        calib_frame_count = 0
        for fname in FINGERS.keys():
            calib_data[fname] = {'open_samples': [], 'closed_samples': [], 'open': None, 'closed': None}
            smooth_buffers[fname].clear()
        print("\n✓ Recalibrating...")

# Cleanup
print("\n✓ Shutting down...")
cap.release()
cv2.destroyAllWindows()
arduino.close()
print("✓ Done")
